In [37]:
import pandas as pd
import numpy as np
from datetime import timedelta
from scipy.fftpack import fft, ifft,rfft
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from joblib import dump, load


In [38]:
data=pd.read_csv('test.csv',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,43,46,49,58,71,85,91,89,80,76,...,128,126,122,112,93,91,91,84,74,67
1,84,86,88,89,108,125,136,166,207,267,...,284,284,284,284,280,268,264,266,264,279
2,266,264,279,292,303,312,312,307,288,275,...,200,183,173,167,158,148,139,131,123,112
3,67,65,67,73,74,73,72,72,75,77,...,122,126,126,129,133,136,138,138,135,139
4,179,173,174,180,184,180,168,161,169,189,...,210,209,208,220,239,226,217,211,205,195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,198,190,185,187,190,195,201,203,190,187,...,187,188,188,188,187,193,198,201,196,184
227,187,190,195,201,203,190,187,188,189,190,...,188,187,193,198,201,196,184,172,169,171
228,141,140,143,144,145,147,149,153,160,168,...,236,245,264,284,335,340,368,385,399,400
229,226,230,236,245,264,284,335,340,368,385,...,400,400,400,400,400,400,400,400,400,400


In [39]:
def createnomealfeaturematrix(non_meal_data):
    index_to_remove_non_meal=non_meal_data.isna().sum(axis=1).replace(0,np.nan).dropna().where(lambda x:x>5).dropna().index
    non_meal_data_cleaned=non_meal_data.drop(non_meal_data.index[index_to_remove_non_meal]).reset_index().drop(columns='index')
    non_meal_data_cleaned=non_meal_data_cleaned.interpolate(method='linear',axis=1)
    index_to_drop_again=non_meal_data_cleaned.isna().sum(axis=1).replace(0,np.nan).dropna().index
    non_meal_data_cleaned=non_meal_data_cleaned.drop(non_meal_data_cleaned.index[index_to_drop_again]).reset_index().drop(columns='index')
    non_meal_feature_matrix=pd.DataFrame()
    
    power_first_max=[]
    index_first_max=[]
    power_second_max=[]
    index_second_max=[]
    power_third_max=[]
    for i in range(len(non_meal_data_cleaned)):
        array=abs(rfft(non_meal_data_cleaned.iloc[:,0:24].iloc[i].values.tolist())).tolist()
        sorted_array=abs(rfft(non_meal_data_cleaned.iloc[:,0:24].iloc[i].values.tolist())).tolist()
        sorted_array.sort()
        power_first_max.append(sorted_array[-2])
        power_second_max.append(sorted_array[-3])
        power_third_max.append(sorted_array[-4])
        index_first_max.append(array.index(sorted_array[-2]))
        index_second_max.append(array.index(sorted_array[-3]))
    
    non_meal_feature_matrix['power_second_max']=power_second_max
    non_meal_feature_matrix['power_third_max']=power_third_max
    first_differential_data=[]
    second_differential_data=[]
    standard_deviation=[]
    for i in range(len(non_meal_data_cleaned)):
        first_differential_data.append(np.diff(non_meal_data_cleaned.iloc[:,0:24].iloc[i].tolist()).max())
        second_differential_data.append(np.diff(np.diff(non_meal_data_cleaned.iloc[:,0:24].iloc[i].tolist())).max())
        standard_deviation.append(np.std(non_meal_data_cleaned.iloc[i]))
    non_meal_feature_matrix['2ndDifferential']=second_differential_data
    non_meal_feature_matrix['standard_deviation']=standard_deviation
    return non_meal_feature_matrix


In [40]:
dataset=createnomealfeaturematrix(data)

In [36]:
from joblib import dump, load
with open('model.pickle', 'rb') as pre_trained:
    pickle_file = load(pre_trained)
    predict = pickle_file.predict(dataset)    
    pre_trained.close()

In [33]:
res = pd.DataFrame(predict)

res.to_csv('Result.csv',index=False,header=False)

In [35]:
#np.mean(res)